In [55]:
from langchain.agents import create_sql_agent,create_openai_functions_agent,create_openapi_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
import os
import openai
from langchain_community.llms import AzureOpenAI
from langchain.agents.agent_types import AgentType
os.environ["OPENAI_API_TYPE"] = "azure"
import openai



In [3]:
openai.api_type = "azure"
openai.api_base = "https://like-card-test.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "85889c7998dd4adb9a4c89abe56b1242"
openai.azure_endpoint="https://like-card-test.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview"

In [56]:
import os

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["OPENAI_API_BASE"] = "https://like-card-test.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "85889c7998dd4adb9a4c89abe56b1242"
os.environ["OPENAI_API_ENDPOINT"]="https://like-card-test.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview"

In [57]:
openai.api_type = "azure"
openai.api_base = "https://like-card-test.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "85889c7998dd4adb9a4c89abe56b1242" 
openai.azure_endpoint="https://like-card-test.openai.azure.com/openai/deployments/gpt-35-turbo-instruct/chat/completions?api-version=2023-07-01-preview"

In [6]:
f = open("Credintials.txt", "r")
email = None
password = None
for i, line in enumerate(f): 
    if i == 0:
        email = line.strip()  # Assign the first line to email variable
    elif i == 1:
        password = line.strip()  # Assign the second line to password variable

f.close()

In [7]:
server = 'anmx3c2yurjethctgyba6xtuaq-vkvbn7t4322edeguc3ufhjtjeu.datawarehouse.pbidedicated.windows.net'
database = 'likeCard'
driver='ODBC Driver 18 for SQL Server'
user= email # use %40 inplace of @
pass1=password # use %40 inplace of @   
db_string =f'mssql+pyodbc://{user}:{pass1}@{server}/{database}?driver={driver}&Trusted_Connection=no&Authentication=ActiveDirectoryInteractive'
dp=SQLDatabase.from_uri(db_string)


In [8]:
llm = AzureOpenAI( model_name="gpt-35-turbo-instruct",deployment_name="gpt-35-turbo-instruct",temperature=0.2)

c:\Users\AbdelKaderTalaat\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `AzureOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.AzureOpenAI instead.
  warn_deprecated(
c:\Users\AbdelKaderTalaat\anaconda3\Lib\site-packages\langchain_community\llms\openai.py:875: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://like-card-test.openai.azure.com/ to https://like-card-test.openai.azure.com/openai.
  warnings.warn(
c:\Users\AbdelKaderTalaat\anaconda3\Lib\site-packages\langchain_community\llms\openai.py:882: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.wa

In [9]:
toolkit = SQLDatabaseToolkit(db=dp,llm=llm)


In [10]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True ,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,

)

In [11]:
from langchain.memory import ConversationBufferMemory


In [12]:
SQL_SUFFIX = """Begin!
history: {history}
Task: Generate a concise SQL Server query using only function calls and parentheses to accurately answer the question: {query}
Thought: 
-Identify correct tables: Determine the tables containing the related information.
-Join tables: Join these tables based on the appropriate relationships if needed.
-Aggregate: Use functions like SUM() and GROUP BY for calculations if needed.
-Select relevant columns.
-Return the final answer as a string: execute the query and return the answer in a structured format

{agent_scratchpad} 
"""

In [13]:
agent_executor = create_sql_agent(   
    llm,
    SQLDatabaseToolkit(db=dp, llm=llm),
    agent_executor_kwargs={
        "memory": ConversationBufferMemory(
            input_key="query", memory_key="history", return_messages= True

        )
    },
    
    suffix=SQL_SUFFIX,
    input_variables=["query", "history", "agent_scratchpad"],
    verbose= True
)  

In [14]:
query = "what is the sales in each country"

In [19]:
output=agent_executor.invoke(query)



> Entering new AgentExecutor chain...
Action: sql_db_query_checker
Action Input: SELECT SUM(sales) AS sales, country FROM sales_table GROUP BY country
Observation: 
SELECT SUM(sales) AS sales, country FROM sales_table GROUP BY country
Thought:-Check if the query is correct: Use sql_db_query_checker to check if the query is correct.
-If the query is incorrect, rewrite the query and try again.

Action: sql_db_query
Action Input: SELECT SUM(sales) AS sales, country FROM sales_table GROUP BY country
Observation: Error: (pyodbc.ProgrammingError) ('42S02', "[42S02] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Invalid object name 'sales_table'. (208) (SQLExecDirectW)")
[SQL: SELECT SUM(sales) AS sales, country FROM sales_table GROUP BY country]
(Background on this error at: https://sqlalche.me/e/14/f405)
Thought:-The error message indicates that the table 'sales_table' does not exist.
-Use sql_db_schema to query the correct table fields.

Action: sql_db_schema
Action Input: sales_t

In [30]:
output['output']

'Saudi Arabia: 4509132904.81992, United Arab Emirates: 277380670.11894894, Kuwait: 295277868.8488867, Jordan: 77439333.78599454, Bahrain: 6782407.03299998, Oman: 54535725.93599716, Qatar: 49046679.49999773, Lebanon: 74674590.77301763, Egypt: 6848128.727999887, Iraq: 3600636.6919999965'

In [66]:

def graph_code_generation(model_answer):
    prompt = """Begin!
    Task: convert the following text to dataframe {model_answer} using python
    and create a python code to vizualize this dataframe
    output: should be python code only
    """

    # Create agent executor with prompt and tools
    agent_executor = OpenAI(
        engine="gpt-35-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ],
                stop=None,
       
    )
 

    graph = agent_executor
    
    return graph

print(graph_code_generation(output['output']))


c:\Users\AbdelKaderTalaat\anaconda3\Lib\site-packages\langchain_core\utils\utils.py:159: UserWarning: WARNING! engine is not default parameter.
                engine was transferred to model_kwargs.
                Please confirm that engine is what you intended.
  warnings.warn(
c:\Users\AbdelKaderTalaat\anaconda3\Lib\site-packages\langchain_core\utils\utils.py:159: UserWarning: WARNING! messages is not default parameter.
                messages was transferred to model_kwargs.
                Please confirm that messages is what you intended.
  warnings.warn(
c:\Users\AbdelKaderTalaat\anaconda3\Lib\site-packages\langchain_core\utils\utils.py:159: UserWarning: WARNING! stop is not default parameter.
                stop was transferred to model_kwargs.
                Please confirm that stop is what you intended.
  warnings.warn(


OpenAI
Params: {'model_name': 'gpt-3.5-turbo-instruct', 'temperature': 0.7, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 256, 'engine': 'gpt-35-turbo', 'messages': [{'role': 'user', 'content': 'Begin!\n    Task: convert the following text to dataframe {model_answer} using python\n    and create a python code to vizualize this dataframe\n    output: should be python code only\n    '}], 'stop': None}


In [61]:
openai.__version__

'1.7.0'

In [24]:
PROMPT = """ 
Given an input question, first create sql server query to run,  
then look at the results of the query second creat python code to run  a graph of the result then  return the answer.  
The question: {question}
"""
question = "what is top 5 products names in terms of amount in each country"

In [25]:
out=agent_executor.invoke(question)

print(out['output'])



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: BL_Mapping_Table, Boost_customers, Customers_DAX_table, DAX_table, GMV_Calender_DAX_table, GMV_DimDate_DAX_table, Same_period, dimDate, dimDateEX, dim_B2B_Sales_Channel, dim_Target, dim_allusers, dim_balance, dim_categories, dim_industry, dim_log_status, dim_parent_categories, dim_payment_methods, dim_pos_sales_channel, dim_products, dim_segments, dim_stores, dim_users_b2b, dim_users_b2c, dim_users_pos, exploration, fact_b2b_trans, fact_b2c_trans, fact_points, fact_pos_trans, fact_salla_trans, fact_transactions, fact_vouchers
Thought:I should query the schema of the most relevant tables, and please return the output as a string.
Action: sql_db_schema
Action Input: dim_products
Observation: 
CREATE TABLE dim_products (
	product_id INTEGER NOT NULL, 
	product_name VARCHAR(250) COLLATE Latin1_General_100_BIN2_UTF8 NULL, 
	category_id INTEGER NULL, 
	CONSTRAINT [PK_product_id] PRIMARY KEY (prod

In [22]:
print(out['output'])

NameError: name 'out' is not defined

In [18]:
from langchain import hub
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

In [ ]:
#--

In [19]:
print(out['output'])

I don't know


In [20]:
 AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

NameError: name 'agent' is not defined